<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch9_8_pg_607~624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08 파이썬 추천 시스템 패키지 - Surprise

# Surprise 패키지 소개
- 앞에서 다룬 예제 코드는 최적화나 수행 속도 측면에서 좀 더 보완이 필요
- 다양한 추천 알고리즘 (사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링)을 쉽게 적용해 추천 시스템 구축
- 사이킷런의 핵심 API와 유사한 API명으로 작성: fit, predict(), train_test_split, cross_validate, gridsearchcv 등의 클래스를 통해 추천 시스템을 위한 모델 셀렉션 평가, 하이퍼 파라미터 튜닝 등의 기능 제공.

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095437 sha256=5edf321f73e030dd060484fd7ab5686efe759704d803b88597aaa817237ee1a5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Surprise를 이용한 추천 시스템 구축
- 예제는 추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행

In [5]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# 추천을 위한 데이터 세트 로딩
- 사용자-영화 평점 데이터 포맷과 같이 주요 데이터가 row 레벨 형태로 돼있는 포맷의 데이터만 처리함
- 과거 버전의 데이터, 최근 영화에 대한 평점 없다.
- 테스트와 트레인 데이터셋으로 분리
- 탭으로 구분되어 있음

In [3]:
data=Dataset.load_builtin('ml-100k')
#수행시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset=train_test_split(data,test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


# SVD로 잠재요인 협업 필터링 수행

In [4]:
algo=SVD() #알고리즘 객체 생성
algo.fit(trainset) # 이 알고리즘에 fit을 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습

- 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천 수행
- test / predict : 추천을 예측하는 메서드 -> 입력된 데이터 세트에 대하 추천 데이터 세트를 만들어줌
- preduct: 개별 사용자와 영화에 대하 추천 평점을 반환

In [5]:
# test(): 테스트 데이터 세트 전체에 대해 추천 영화 평점 데이터를 생성한 뒤 최초 5개만 추출하는 예제
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5168541619891633, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7279216574141842, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.191149974330706, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.260332330422432, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2528737010663606, details={'was_impossible': False})]

- 호출 결과는 파이썬 리스트이며, 크기는 입력 인자 데이터 세트의 크기와 같은 prediction 객체를 내부에 갖고 있다
- Prediction 객체는 Surprise 패키지에서 제공하는 데이터 타입: 개별 사용자 아이디, 영화 아이디, 실제 평점 정보에 기반해 Surprise의 추천 예측 평점 데이터를 튜플 형태로 갖고 있음
- Prediction 객체 detail 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용
- was_impossible이 True이면, 예측값을 생성할 수 없는 데이터


#리스트 객체에 내포된 Prediction 객체의 userid, itemid, est 등의 속성에 접근

In [7]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5168541619891633),
 ('882', '291', 3.7279216574141842),
 ('535', '507', 4.191149974330706)]

# predict()를 이용한 추천 예측
- 개별 사용자의 아이템에 대한 추천 평점을 예측
- 인자로 개별 사용자 아이디, 아이템 아이디 입력 -> 추천 예측 평점을 포함한 정보 반환

In [8]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.29   {'was_impossible': False}


- 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환
- test() 메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

# 테스트 데이터 세트를 이용한 추천 예측 평점과 실제 평점 차이 평가

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9476


0.9475908416166936

# Surprise 주요 모듈 소개

Dataset

- 로우 레벨로된 데이터 세트만 적용 가능하므로, 네번째 칼럼부터는 아예 로딩을 수행하지 않음 
- 따라서 데이터 세트의 칼럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순으로 되어 있어야 한다.

# OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 로딩 되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안됨 
- ratings 파일에 있으므로 to_csv() 함수로 헤더를 삭제하자.

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ESAA/2023-1/필사/data/grouplens/") # working directory를 설정함.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd

ratings=pd.read_csv('ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

# 데이터 파일의 파싱 포맷 정의
- reader 객체 생성 시 line_foramt 인자로 user, item, rating, timestamp의 4개 칼럼으로 데이터가 구성되어 있음을 명시. 
- 각 칼럼의 분리 문자는 콤마, 평점 단위는 0.5, 최대 평점은 5로 설정. 
- reader 설정을 완료하면 이를 기반으로 데이터를 파싱하며 로딩.

In [14]:
from surprise import Reader

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('ratings_noh.csv', reader=reader)

- surprise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우 Reader 클래스를 먼저 설정해야 함
- Reader 클래스의 주요 생성 파라미터는 다음과 같다

1. line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식.
2. sep(char): 칼럼을 분리하는 분리자, 디폴트는  탭
3. rating_scale(tuple, optional): 평점 값의 최소~최대 평점을 설정, 디폴트는 (1,5) 이지만, 우리 파일이 경우 최대 최소가 위와 같으므로 위와 같이 설정


#  SVD 행렬 분해 기법을 이용해 추천을 예측

In [ ]:
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

#수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

- 얘 때문에 내 램이 다운됨

# 판다스 DF에서 Surprise 데이터 세트로 로딩
- 주의할 점: 사용자 아이디, 아이템 아이디, 평점 칼럼 순서로 되어 있어야 함

In [6]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

#ratings df에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7820


0.7819971825709526

# Surprise 추천 알고리즘 클래스

- 추천 예측을 위해 자주 사용되는 추천 알고리즘 클래스: SVD, KNNBasic, BaselineOnly
- surprise SVD의 비용 함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 regularization을 적용
- SVD 클래스의 입력 파라미터: n_factors와 n_epochs의 값을 변경해 튜닝할 수 있으나 튜닝 효과는 크지 않다.
- 추천 알고리즘 예측 성능 벤치마크 결과: SVD++ 알고리즘의 성적이 가장 좋지만, 시간이 오래 걸린다. SVD++을 제외하면 SVD, k-NN baseline이 가장 성적이 좋다.

# 베이스라인 평점
- 개인의 성형을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것
- 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산
- 사용자 편향 점수=사용자별 아이템 평점 평균 값 - 전체 평균 평점
- 아이템 편향 점수=아이템별 평점 평균 값 - 전체 평균 평점

# 교차 검증과 하이퍼 파라미터 튜닝
- cross_validate()
- gridsearchcv()

# cross_validate

In [ ]:
from surprise.model_selection import cross_validate

#판다스 df에서 surprise 데이터 세트로 데이터 로딩
ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

- cross_validate(): 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여줌

#gridsearchcv

In [ ]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid={'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

#CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 gridsearchcv 구성
gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# Surprise를 이용한 개인화 영화 추천 시스템 구축
- surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천 
-> ratings 데이터를 분리하지 않고 전체를 학습 데이터로 사용

- surprise는 데이터 세트를 split()을 이용해 내부에서 사용하는 trainset 클래스 객체로 변환하지 않으면 fit으로 학습할 수가 없어, DatasetAutoFolds이용

In [ ]:
# 이 코드는 오류 발생

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

# DatasetAutoFolds

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#datasetaufofolds 클래스를 ratings_noh.csv 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

#build_full_trainset을 이용해 생성된 학습 데이터를 기반으로 학습 수행

- SVD

In [ ]:
algo=SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- userID =9인 사용자를 지정해 아직 평점을 주지 않은 영화 movieID 42로 선정한 뒤 평점을 계산

In [ ]:
#영화에 대한 상세 속성 정보 df 로딩
movies=pd.read_csv('movies.csv')

#userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

# predict() 이용해 예측

In [ ]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

- 사용자가 평점을 매기지 않은 전체 영화를 추출 -> 예측 평점 순으로 영화 추천

# 아직 평점을 매기지 않은 영화 정보 반환: get_unseen_surprise()

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  #모든 영화의 movieId를 리스트로 생성
  total_movies=movies['movieId'].tolist()

  #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

# 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 예측 평점을 가진 순으로 영화를 추천

In [ ]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있다.
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  #이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의
  #sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  #sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  #top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds=[ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])